# Knapsack

$$
\begin{align}
    \text{max} & \quad \sum_{i \in I} c_{i} x_{i}\\
    \text{s.t.} & \quad \sum_{i \in I} w_{i} x_{i} \leq Q\\
    & x_{i} \in \{0, 1\} & \forall i \in I
\end{align}
$$

In [1]:
import pyomo.environ as pyo

In [2]:
# Define input data
costs = {
"A": 4,
"B": 2,
"C": 10,
"D": 5,
"E": 6,
}

weights = {
"A": 3,
"B": 2,
"C": 7,
"D": 4,
"E": 5,
}

capacity = 11

In [3]:
# Instantiate model as a Pyomo ConcreteModel
model = pyo.ConcreteModel()

In [4]:
# Sets
model.I = pyo.Set(initialize=costs.keys())

In [5]:
# Parameters
model.c = pyo.Param(model.I, initialize=costs)
model.w = pyo.Param(model.I, initialize=weights)
model.Q = pyo.Param(initialize=capacity)

In [6]:
# Variables
model.x = pyo.Var(model.I, within=pyo.Binary)

In [7]:
# Constraints
def capacity_cstr(model):
    return sum(model.w[i] * model.x[i] for i in model.I) <= model.Q


model.cap_cstr = pyo.Constraint(rule=capacity_cstr)

In [8]:
# Objective
def obj(model):
    return sum(model.c[i] * model.x[i] for i in model.I)


model.obj = pyo.Objective(rule=obj, sense=pyo.maximize)

In [9]:
model.I.display()
print('_'*60)
model.x.display()
print('_'*60)
model.w.display()
print('_'*60)
model.c.display()
print('_'*60)
model.Q.display()

I : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    5 : {'A', 'B', 'C', 'D', 'E'}
____________________________________________________________
x : Size=5, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      A :     0 :  None :     1 : False :  True : Binary
      B :     0 :  None :     1 : False :  True : Binary
      C :     0 :  None :     1 : False :  True : Binary
      D :     0 :  None :     1 : False :  True : Binary
      E :     0 :  None :     1 : False :  True : Binary
____________________________________________________________
w : Size=5, Index=I, Domain=Any, Default=None, Mutable=False
    Key : Value
      A :     3
      B :     2
      C :     7
      D :     4
      E :     5
____________________________________________________________
c : Size=5, Index=I, Domain=Any, Default=None, Mutable=False
    Key : Value
      A :     4
      B :     2
      C :    10
      D :     5
      E :  

In [10]:
# Instantiate Highs persistent solver (make sure highspy is installed)
# solver = pyo.SolverFactory("appsi_highs")
solver = pyo.SolverFactory("appsi_highs")

In [11]:
solver.set_instance(model)

In [12]:
# Apply method solve
solver.solve(model)

{'Problem': [{'Lower bound': 15.0, 'Upper bound': 15.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [13]:
# Create output dictionary from variables
model.x.extract_values()

{'A': -0.0, 'B': 0.0, 'C': 1.0, 'D': 1.0, 'E': 0.0}